# 00F — Monetary & Banking Analytics

**Purpose**: Derive "Central Banker" insights: Credit Impulse, Money Multiplier, and Fiscal Dominance.

**Key Metrics**:
1.  **Credit Impulse**: The *change in the change* of new credit flow. (Leads GDP by 6-9 months).
2.  **Money Multiplier**: `M3 / Reserve Money`. (Banking efficiency).
3.  **Fiscal Dominance Ratio**: `Net RBI Credit to Govt / Total Reserve Money`.

**Outputs**:
- `../data_processed/india_monetary_monthly.parquet`

---

## 1. Load RBI Data

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

PROCESSED_PATH = Path('../data_processed')
RBI_PATH = PROCESSED_PATH / 'rbi_macro_all_long.parquet'

# Load RBI Data
if RBI_PATH.exists():
    df = pd.read_parquet(RBI_PATH)
    print(f"Loaded {len(df):,} rows from RBI data")
else:
    raise FileNotFoundError("RBI data missing. Run 00B.")

Loaded 41,884 rows from RBI data


## 2. Extract Core Series

Looking for:
- **M3** (Broad Money)
- **M1** (Narrow Money)
- **Reserve Money** (M0 / Base Money)
- **Credit to Commercial Sector** (Private Credit)
- **Net RBI Credit to Government** (Fiscal Financing)
- **Credit Deposit Ratio** (Banking Tightness)

In [2]:
# Define targets (UPDATED WITH CORRECT MAPPINGS)
MONETARY_VARS = {
    'M3': ['M3'],
    'M1': ['M1'],
    'RESERVE_MONEY': ['RBI BALANCE SHEET-LIABILITIES', 'RBI BALANCE SHEET-CURRENCY', 'Reserve Money', 'M0'], # Proxy: Total Liability ~ Base Money
    'CREDIT_COMMERCIAL': ['CREDIT TO THE COMMERCIAL SECTOR', 'CREDIT TO THE COMMERCIAL SECTOR BY THE BANKING SYSTEM'],
    'CREDIT_GOVT': ['RBI BALANCE SHEET-LOANS-GOVERNMENTS', 'NET RBI CREDIT TO GOVERNMENT'],
    'CD_RATIO': ['Credit Deposit Ratio (%)', 'Credit-Deposit Ratio']
}

# Helper to find available series matching patterns
available_series = df['series_name'].unique()
mapped_series = {}

for key, patterns in MONETARY_VARS.items():
    found = False
    for pattern in patterns:
        match = next((s for s in available_series if pattern.lower() in s.lower()), None)
        if match:
            mapped_series[key] = match
            found = True
            break
    if not found:
        print(f"⚠️ Warning: No match found for {key}")

print("mapped_series:", mapped_series)

# Filter and Pivot
monetary_df = df[df['series_name'].isin(mapped_series.values())].copy()
monetary_df['key'] = monetary_df['series_name'].map({v: k for k, v in mapped_series.items()})

monetary_wide = monetary_df.pivot_table(index='Date', columns='key', values='value')
monetary_monthly = monetary_wide.resample('ME').last().ffill()

print("\nMonetary Data Shape:", monetary_monthly.shape)

mapped_series: {'M3': 'M3', 'M1': 'M1', 'RESERVE_MONEY': 'RBI BALANCE SHEET-LIABILITIES', 'CREDIT_COMMERCIAL': 'CREDIT TO THE COMMERCIAL SECTOR BY THE BANKING SYSTEM', 'CREDIT_GOVT': 'RBI BALANCE SHEET-LOANS-GOVERNMENTS', 'CD_RATIO': 'Credit Deposit Ratio (%)'}

Monetary Data Shape: (899, 6)


## 3. Calculate Advanced Metrics

In [3]:
# 1. Money Multiplier (M3 / Reserve Money)
# Note: Need to look for a proxy if Reserve Money series is missing or ambiguous.
# For now, we calculate if both exist.
if 'M3' in monetary_monthly.columns and 'RESERVE_MONEY' in monetary_monthly.columns:
    monetary_monthly['MONEY_MULTIPLIER'] = monetary_monthly['M3'] / monetary_monthly['RESERVE_MONEY']

# 2. Fiscal Dominance (RBI funding Govt vs Private)
if 'CREDIT_GOVT' in monetary_monthly.columns and 'CREDIT_COMMERCIAL' in monetary_monthly.columns:
    monetary_monthly['FISCAL_DOMINANCE_RATIO'] = monetary_monthly['CREDIT_GOVT'] / monetary_monthly['CREDIT_COMMERCIAL']

# 3. Credit Impulse (2nd Derivative of Credit Stock)
# Formula: Change in Net Lending / GDP (Approximated by Credit Stock 2nd diff normalized)
if 'CREDIT_COMMERCIAL' in monetary_monthly.columns:
    # 1st Deriv: New Credit Flow (MoM)
    monetary_monthly['CREDIT_FLOW'] = monetary_monthly['CREDIT_COMMERCIAL'].diff()
    # 2nd Deriv: Change in Flow (Impulse)
    # We normalize by the 12M moving average of the stock to proxy for economy size scaling
    scaler = monetary_monthly['CREDIT_COMMERCIAL'].rolling(12).mean()
    monetary_monthly['CREDIT_IMPULSE'] = monetary_monthly['CREDIT_FLOW'].diff(3).rolling(3).mean() / scaler * 100

print("Computed Metrics:")
cols_to_show = [c for c in ['MONEY_MULTIPLIER', 'FISCAL_DOMINANCE_RATIO', 'CREDIT_IMPULSE'] if c in monetary_monthly.columns]
print(monetary_monthly[cols_to_show].tail() if cols_to_show else "Data insufficient")

Computed Metrics:
key         MONEY_MULTIPLIER  FISCAL_DOMINANCE_RATIO  CREDIT_IMPULSE
Date                                                                
2025-09-30          3.503082                0.001656        0.308565
2025-10-31          3.671706                0.000496       33.330804
2025-11-30          3.681331                0.000933        0.840759
2025-12-31          3.597322                0.001106        1.591946
2026-01-31          3.561633                0.001115      -63.249060


## 4. Normalize (Z-Scores) for Regime Analysis

In [4]:
def rolling_z(series, window=36): # 3 Years
    return (series - series.rolling(window, min_periods=12).mean()) / series.rolling(window, min_periods=12).std()

for col in ['MONEY_MULTIPLIER', 'FISCAL_DOMINANCE_RATIO', 'CREDIT_IMPULSE', 'CD_RATIO']:
    if col in monetary_monthly.columns:
        monetary_monthly[f'{col}_ZSCORE'] = rolling_z(monetary_monthly[col])

print("Final Monetary Regimes:")
print(monetary_monthly.filter(like='_ZSCORE').tail())

Final Monetary Regimes:
key         MONEY_MULTIPLIER_ZSCORE  FISCAL_DOMINANCE_RATIO_ZSCORE  \
Date                                                                 
2025-09-30                -0.639596                       1.188062   
2025-10-31                -0.539078                      -0.852491   
2025-11-30                -0.535378                      -0.119622   
2025-12-31                -0.586861                       0.157170   
2026-01-31                -0.608775                       0.179099   

key         CREDIT_IMPULSE_ZSCORE  CD_RATIO_ZSCORE  
Date                                                
2025-09-30               0.609671         0.727628  
2025-10-31               5.809250         0.684302  
2025-11-30              -0.016779         0.642045  
2025-12-31               0.112460         0.599816  
2026-01-31              -5.173891         0.557634  


## 5. Export

In [5]:
if not monetary_monthly.empty:
    output_path = PROCESSED_PATH / 'india_monetary_monthly.parquet'
    monetary_monthly.to_parquet(output_path)
    print(f"✓ Saved: {output_path}")
else:
    print("No monetary data generated.")

✓ Saved: ..\data_processed\india_monetary_monthly.parquet
